In [ ]:
#r "nuget:Microsoft.Spark"
using System;
using System.Collections.Generic;
using Microsoft.Spark.Sql;
using Microsoft.Spark.Sql.Types;

Environment.SetEnvironmentVariable("EVENT_HUB_CONNECTION_STRING", "Endpoint=sb://evhns-timeseries-timeseries-u-001.servicebus.windows.net/;SharedAccessKeyName=listen;SharedAccessKey=lGW5ENgWGFeRhNNBhPkq4uAYrebkWhSTa0ZbvafB9cw=;EntityPath=received-timeseries");
/////////////////////////////////////////////////////

var spark = SparkSession.Builder().GetOrCreate();
var timeseries_unprocessed_path = "TODO";

var eventHubConnectionString = Environment.GetEnvironmentVariable("EVENT_HUB_CONNECTION_STRING");

// TODO BJARKE: Extract hostname from connection string: Endpoint=sb://evhns-integrationstest-u-002.servicebus.windows.net/;SharedAccessKeyName=RootManageSharedAccessKey;SharedAccessKey=*******
string bootstrapServers = "evhns-timeseries-timeseries-u-001.servicebus.windows.net:9093"; // 9093 is the port used to communicate with Event Hubs, see [troubleshooting guide](https://docs.microsoft.com/azure/event-hubs/troubleshooting-guide)
string eh_sasl = $"org.apache.kafka.common.security.plain.PlainLoginModule required username=\"$ConnectionString\" password=\"{eventHubConnectionString}\";";
Console.WriteLine(eh_sasl);

var streamingDf = spark
    .ReadStream()
    .Format("kafka")
    .Option("kafka.bootstrap.servers", bootstrapServers)
    .Option("subscribe", "received-timeseries")
    .Option("kafka.sasl.mechanism", "PLAIN")
    .Option("kafka.security.protocol", "SASL_SSL")
    .Option("kafka.sasl.jaas.config", eh_sasl)
    .Option("kafka.request.timeout.ms", "60000")
    .Option("kafka.session.timeout.ms", "60000")
    .Option("failOnDataLoss", "false")
    .Load();

    var checkpointPath = "received_time_series_checkpoint";
    //var checkpointPath = $"abfss://{delta_lake_container_name}@{storage_account_name}.dfs.core.windows.net/checkpoint"
     streamingDf
         .WriteStream()
    //     //.Option("checkpointLocation", checkpointPath)
    //     // TODO: Trigger setup
         .ForeachBatch((df, epochId) => ProcessEventhubItem(df, epochId, timeseries_unprocessed_path))
         .Start().AwaitTermination();

  //  var dfWrite = streamingDf.WriteStream().OutputMode("append").Format("console").Start();
    //dfWrite.AwaitTermination();
    //spark.Stop();
   
    public static void ProcessEventhubItem(DataFrame df, long epochId, string timeseriesUnprocessedPath)
    {
        Console.WriteLine("Hello");
        if(df.Head(1).Count()>0){
            df.Show();
        }
        // if (df == null) throw new ArgumentNullException(nameof(timeseriesUnprocessedPath));
        // df = df
        //     .WithColumn("year", Functions.Year(df["enqueuedTime"]))
        //     .WithColumn("month", Functions.Month(df["enqueuedTime"]))
        //     .WithColumn("day", Functions.DayOfMonth(df["enqueuedTime"]));
        // // df
        // //     .Write()
        // //     .PartitionBy("year", "month", "day")
        // //     .Format("delta")
        // //     .Mode(SaveMode.Append)
        // //     .Save(timeseries_unprocessed_path);
        // df.Show();
    }

Installed Packages Microsoft.Spark, 2.1.0

Loading extensions from `Microsoft.Data.Analysis.Interactive.dll`

[2022-03-01T13:33:37.2244035Z] [02e1cd668e4c] [Info] [ConfigurationService] 'DOTNETBACKEND_PORT' environment variable is not set.
[2022-03-01T13:33:37.2266517Z] [02e1cd668e4c] [Info] [ConfigurationService] Using port 5567 for connection.
[2022-03-01T13:33:37.2272630Z] [02e1cd668e4c] [Info] [JvmBridge] JvMBridge port is 5567
[2022-03-01T13:33:37.2279088Z] [02e1cd668e4c] [Info] [JvmBridge] The number of JVM backend thread is set to 10. The max number of concurrent sockets in JvmBridge is set to 7.
org.apache.kafka.common.security.plain.PlainLoginModule required username="$ConnectionString" password="Endpoint=sb://evhns-timeseries-timeseries-u-001.servicebus.windows.net/;SharedAccessKeyName=listen;SharedAccessKey=lGW5ENgWGFeRhNNBhPkq4uAYrebkWhSTa0ZbvafB9cw=;EntityPath=received-timeseries";
[2022-03-01T13:33:41.6816339Z] [02e1cd668e4c] [Info] [CallbackServer] Starting CallbackServer.
[2022-03-01T13:33:41.7116472Z] [02e1cd668e4c] [Info] [CallbackServer] Started CallbackServer on 127.0.0.1:4

Error: System.Exception: JVM method execution failed: Nonstatic method 'awaitTermination' failed for class '19' when called with no arguments
 ---> Microsoft.Spark.JvmException: org.apache.spark.sql.streaming.StreamingQueryException: Error while verifying end of stream.
=== Streaming Query ===
Identifier: [id = be3dddbe-9aa7-414e-b437-5173f26b9e5c, runId = 54af6c6f-131e-4ff3-b118-da3963cea92e]
Current Committed Offsets: {}
Current Available Offsets: {KafkaV2[Subscribe[received-timeseries]]: {"received-timeseries":{"2":8,"1":10,"3":18,"0":14}}}

Current State: ACTIVE
Thread State: RUNNABLE

Logical Plan:
StreamingDataSourceV2Relation [key#7, value#8, topic#9, partition#10, offset#11L, timestamp#12, timestampType#13], org.apache.spark.sql.kafka010.KafkaSourceProvider$KafkaScan@6d025ea5, KafkaV2[Subscribe[received-timeseries]]

	at org.apache.spark.sql.execution.streaming.StreamExecution.org$apache$spark$sql$execution$streaming$StreamExecution$$runStream(StreamExecution.scala:325)
	at org